# `asyncio`

1. What is (and isn't?) `asyncio`
2. How it works, using non-`asyncio` stuff
3. Coroutines and tasks and `async def`
4. Running coroutines and `await`
5. Task groups
6. Getting results (and exceptions)
7. Task pools
8. Retrieving URLs
9. Chat server

# The background

Two related terms in programming:

- Concurrency -- a more general term meaning that we can benefit from pieces of our program executing semi-independently, even if they're not truly indepent of one another
- Parallelism -- running multiple parts of our program in parallel

